# Introduction

EUCLID is a method for discovering material models from heterogeneous data. In this notebook, we will use EUCLID to discover hyperelastic material models from synthetically generated FEM data. Using synthetically generated data has the advantantage that we know the ground truth material model used for generating the data and that we can control the noise level of the data. The full EUCLID code for discovering hyperelastic material models is available under https://github.com/EUCLID-code/EUCLID-hyperelasticity. Here, however, we will not consider the full code, but consider a minimal working example.


```
#    ________ ___  ___ _______ ___     ___ ______
#   /  _____//  / /  //  ____//  /    /  //  _   \
#  /  _____//  /_/  //  /___ /  /___ /  //  /_|  |
# /_______//_______//______//______//__//_______/
# Efficient Unsupervised Constitutive Law Identification and Discovery
# https://euclid-code.github.io/
```



# Load python libraries




In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import torch

# Global variables


In [ ]:
dim = 2
numNodesPerElement = 3
voigtMap = [[0,1],[2,3]]
linkToData = 'https://raw.githubusercontent.com/EUCLID-code/EUCLID-hyperelasticity/refs/heads/main/FEM_data/plate_hole_1k/plate_hole_1k_NeoHookeanJ2/'
loadsteps = [10,20,30,40]
lambda_r = 100

# Classes for organizing data

In [ ]:
class Reaction:
    def __init__(self, dofs, force):
        self.force = force
        self.dofs = dofs

# Subroutines for computing kinematic quantities
Note that plane strain is assumed. This means that F13 = F23 = 0 and F33 = 1. We do not explicitly implement these values.

In [ ]:
def computeCauchyGreenStrain(F):
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]
    C11 = F11**2 + F21**2
    C12 = F11*F12 + F21*F22
    C21 = F11*F12 + F21*F22
    C22 = F12**2 + F22**2
    C = torch.cat((C11,C12,C21,C22),dim=1)
    return C

def computeJacobian(F):
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]
    J = F11*F22 - F12*F21
    return J

def computeStrainInvariants(C):
    C11 = C[:,0:1]
    C12 = C[:,1:2]
    C21 = C[:,2:3]
    C22 = C[:,3:4]
    I1 = C11 + C22 + 1.0
    I2 = C11 + C22 - C12*C21 + C11*C22
    I3 = C11*C22 - C12*C21
    return I1, I2, I3

def computeStrainInvariantDerivatives(F,i):
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]
    dIdF = torch.zeros(F.shape[0],F.shape[1])
    if(i==1):
        # dI1/dF:
        dIdF = 2.0*F
    elif(i==2):
        # dI2/dF:
        dIdF11 = 2.0*F11 - 2.0*F12*F21*F22 + 2.0*F11*(F22**2)
        dIdF12 = 2.0*F12 + 2.0*F12*(F21**2) - 2.0*F11*F21*F22
        dIdF21 = 2.0*F21 + 2.0*(F12**2)*F21 - 2.0*F11*F12*F22
        dIdF22 = 2.0*F22 - 2.0*F11*F12*F21 + 2.0*(F11**2)*F22
        dIdF = torch.cat((dIdF11,dIdF12,dIdF21,dIdF22),dim=1)
    elif(i==3):
        # dI3/dF:
        J = F11*F22 - F12*F21
        dIdF11 = 2.0*F22 * J
        dIdF12 = -2.0*F21 * J
        dIdF21 = -2.0*F12 * J
        dIdF22 = 2.0*F11 * J
        dIdF = torch.cat((dIdF11,dIdF12,dIdF21,dIdF22),dim=1)
    else:
        raise ValueError('Incorrect invariant index')
    return dIdF

# Subroutines for computing features and their derivatives
Note that, under plane strain, the invariants fulfill the relationship I2 = (I1 + I3 − 1). Therefore, the features and thus the strain energy density function can be considered as functions of I1 and I3 only.

In [ ]:
def computeFeatures(I1, I2, I3):
    K1 = I1 * torch.pow(I3,-1/3) - 3.0
    K2 = (I1 + I3 - 1) * torch.pow(I3,-2/3) - 3.0
    J = torch.sqrt(I3)
    numFeatures = 3
    Q = torch.zeros(I1.shape[0],numFeatures)
    Q[:,0:1] = K1
    Q[:,1:2] = K2
    Q[:,2:3] = (J-1)**2
    return Q

def getNumberOfFeatures():
    return computeFeatures(torch.zeros(1,1),torch.zeros(1,1),torch.zeros(1,1)).shape[1]

def differentiateFeaturesWithInvariants(features,I):
    d_feature_dI = torch.zeros(features.shape[0],features.shape[1])
    for i in range(features.shape[1]):
        temp = torch.autograd.grad(features[:,i:(i+1)],I,torch.ones(I.shape[0],1),create_graph=True,allow_unused=True)[0]
        if(type(temp)!=type(None)):
            d_feature_dI[:,i:(i+1)] = temp
    return d_feature_dI

def assembleFeatureDerivative(d_features_dI1,d_features_dI3,dI1dF,dI3dF,ele):
    d_features_dI1_element = d_features_dI1[ele,:]
    d_features_dI3_element = d_features_dI3[ele,:]
    dI1dF_element = dI1dF[ele,:]
    dI3dF_element = dI3dF[ele,:]
    d_features_dF_element = np.outer(d_features_dI1_element,dI1dF_element)+np.outer(d_features_dI3_element,dI3dF_element)
    return d_features_dF_element

# Subroutines for array manipulations

In [ ]:
def assembleB(gradNa,ele):
    B_element = np.zeros((dim*dim,dim*numNodesPerElement))
    for a in range(numNodesPerElement):
        dN1 = gradNa[a][ele,0]
        dN2 = gradNa[a][ele,1]
        B_element[:,2*a:2*a+2] = np.array([(dN1,0.0),(dN2,0.0),(0.0,dN1),(0.0,dN2)])
    return B_element

def assembleGlobalVector(vector_global,vector_element,connectivity,ele):
    # Adds a vector defined at an element to the corresponding position in the global vector.
    for a in range(numNodesPerElement):
        vector_global[2*connectivity[a][ele]] += vector_element[2*a]
        vector_global[2*connectivity[a][ele]+1] += vector_element[2*a+1]
    return vector_global

def assembleGlobalMatrix(matrix_global,matrix_element,connectivity,ele):
    for a in range(numNodesPerElement):
        matrix_global[2*connectivity[a][ele],:] += matrix_element[2*a,:]
        matrix_global[2*connectivity[a][ele]+1,:] += matrix_element[2*a+1,:]
    return matrix_global

def zipper(matrix):
    vector = np.zeros(2*matrix.shape[0], dtype=type(matrix[0,0]))
    for i in range(matrix.shape[0]):
        vector[2*i] = matrix[i,0:1]
        vector[2*i+1] = matrix[i,1:2]
    return vector

# Load data from GitHub repository and assemble linear equations

An exemplary dataset can be found on GitHub under https://github.com/EUCLID-code/EUCLID-hyperelasticity. We directly load the data from there.

The data was generated using FEM simulations of a plate with a hole made of a hyperelastic material. The data contains information about the mesh, the Gauss points, the displacements and the reaction forces for four time steps.

In [ ]:
A_free = []
b_free = []
A_reaction = []
b_reaction = []

for idx_step, step in enumerate(loadsteps):

    # Step 1: Load data
    print('Load data for the ' + str(idx_step+1) + '. load step...')
    url = linkToData + str(step)
    df = pd.read_csv(url + '/output_nodes.csv',dtype=np.float64)
    x_nodes = torch.tensor(df[['x','y']].values)
    u_nodes = torch.tensor(df[['ux','uy']].values)
    bcs_nodes = torch.tensor(df[['bcx','bcy']].round().astype(int).values)
    dirichlet_nodes = (bcs_nodes!=0)
    df = pd.read_csv(url + '/output_reactions.csv',dtype=np.float64)
    reactions = []
    for i in range(torch.max(bcs_nodes).item()):
        reactions.append(Reaction(bcs_nodes == (i+1),df['forces'][i]))
    df = pd.read_csv(url + '/output_elements.csv',dtype=np.float64)
    connectivity = []
    for i in range(numNodesPerElement):
        connectivity.append(torch.tensor(df['node'+str(i+1)].round().astype(int).tolist()))
    df = pd.read_csv(url + '/output_integrator.csv',dtype=np.float64)
    gradNa = []
    for i in range(numNodesPerElement):
        gradNa.append(torch.tensor(df[['gradNa_node'+str(i+1)+'_x','gradNa_node'+str(i+1)+'_y']].values))
    qpWeights = torch.tensor(df['qpWeight'].values)

    numNodes = x_nodes.shape[0]
    numElements = connectivity[0].shape[0]

    # Step 2: Compute kinematic quantities (deformation gradient, right Cauchy-green stretch, invariants)
    print('Compute kinematic quantities for the ' + str(idx_step+1) + '. load step...')
    u = []
    for i in range(numNodesPerElement):
        u.append(u_nodes[connectivity[i],:])
    F=torch.zeros(numElements,4)
    for a in range(numNodesPerElement):
        for i in range(dim):
            for j in range(dim):
                F[:,voigtMap[i][j]] += u[a][:,i] * gradNa[a][:,j]
    F[:,0] += 1.0
    F[:,3] += 1.0
    J = computeJacobian(F)
    C = computeCauchyGreenStrain(F)
    I1, I2, I3 = computeStrainInvariants(C)
    dI1dF = computeStrainInvariantDerivatives(F,1)
    dI2dF = computeStrainInvariantDerivatives(F,2)
    dI3dF = computeStrainInvariantDerivatives(F,3)

    # Step 3: Compute features and their derivatives
    print('Compute features for the ' + str(idx_step+1) + '. load step...')
    I1.requires_grad = True
    I2.requires_grad = True
    I3.requires_grad = True
    features = computeFeatures(I1, I2, I3)
    d_features_dI1 = differentiateFeaturesWithInvariants(features,I1)
    d_features_dI2 = differentiateFeaturesWithInvariants(features,I2)
    d_features_dI3 = differentiateFeaturesWithInvariants(features,I3)

    # Step 4: Convert torch tensors to numpy arrays
    x_nodes = x_nodes.cpu().detach().numpy()
    u_nodes = u_nodes.cpu().detach().numpy()
    dirichlet_nodes = dirichlet_nodes.cpu().detach().numpy()
    qpWeights = qpWeights.cpu().detach().numpy()
    for i in range(len(reactions)):
        reactions[i].dofs = reactions[i].dofs.cpu().detach().numpy()
    for i in range(len(connectivity)):
        connectivity[i] = connectivity[i].cpu().detach().numpy()
    for i in range(len(gradNa)):
        gradNa[i] = gradNa[i].cpu().detach().numpy()
    F = F.cpu().detach().numpy()
    J = J.cpu().detach().numpy()
    C = C.cpu().detach().numpy()
    I1 = I1.cpu().detach().numpy()
    I2 = I2.cpu().detach().numpy()
    I3 = I3.cpu().detach().numpy()
    dI1dF = dI1dF.cpu().detach().numpy()
    dI2dF = dI2dF.cpu().detach().numpy()
    dI3dF = dI3dF.cpu().detach().numpy()
    features = features.cpu().detach().numpy()
    d_features_dI1 = d_features_dI1.cpu().detach().numpy()
    d_features_dI2 = d_features_dI2.cpu().detach().numpy()
    d_features_dI3 = d_features_dI3.cpu().detach().numpy()

    # Step 5: Assemble linear equations
    print('Assemble linear equations for the ' + str(idx_step+1) + '. load step...')
    numFeatures = features.shape[1]
    LHS = np.zeros((dim*numNodes,numFeatures))
    for ele in range(numElements):
        d_features_dF_element = assembleFeatureDerivative(d_features_dI1,d_features_dI3,dI1dF,dI3dF,ele)
        B_element = assembleB(gradNa,ele)
        LHS_element = np.transpose(B_element).dot(np.transpose(d_features_dF_element)) * qpWeights[ele]
        LHS = assembleGlobalMatrix(LHS,LHS_element,connectivity,ele)

    non_dirichlet_nodes = ~(zipper(dirichlet_nodes)) # ~ can cause errors -> use np.logical_not
    LHS_free = LHS[non_dirichlet_nodes,:]
    RHS_free = np.zeros(LHS_free.shape[0])
    LHS_fix = np.zeros([len(reactions),LHS_free.shape[1]])
    RHS_fix = np.zeros(len(reactions))
    reaction_counter = -1
    for reaction in reactions:
        reaction_counter += 1
        one_vector = np.ones(LHS[zipper(reaction.dofs),:].shape[0])
        LHS_fix[reaction_counter] = np.transpose(one_vector).dot(LHS[zipper(reaction.dofs),:])
        RHS_fix[reaction_counter] = reaction.force

    A_free.append(LHS_free)
    b_free.append(RHS_free)
    A_reaction.append(LHS_fix)
    b_reaction.append(RHS_fix)


Load data for the 1. load step...
Compute kinematic quantities for the 1. load step...
Compute features for the 1. load step...
Assemble linear equations for the 1. load step...
Load data for the 2. load step...
Compute kinematic quantities for the 2. load step...
Compute features for the 2. load step...
Assemble linear equations for the 2. load step...
Load data for the 3. load step...
Compute kinematic quantities for the 3. load step...
Compute features for the 3. load step...
Assemble linear equations for the 3. load step...
Load data for the 4. load step...
Compute kinematic quantities for the 4. load step...
Compute features for the 4. load step...
Assemble linear equations for the 4. load step...


# Concatenate all linear equations

In [ ]:
A_free_all = np.concatenate(A_free, axis=0)
b_free_all = np.concatenate(b_free, axis=0)
A_reaction_all = np.concatenate(A_reaction, axis=0)
b_reaction_all = np.concatenate(b_reaction, axis=0)

A = np.concatenate((A_free_all,np.sqrt(lambda_r)*A_reaction_all), axis=0)
b = np.concatenate((b_free_all,np.sqrt(lambda_r)*b_reaction_all), axis=0)

# EUCLID

In [ ]:
lasso = linear_model.Lasso(alpha=0.0001)
lasso.fit(A,b)
print(lasso.coef_)

[0.49835751 0.         1.49943368]
